# Load data with pickle files

In [1]:
# Load the data
import pickle
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as netcdf
import torch

with open('ssp585_time_series.pkl', 'rb') as f:
    dic_ssp585 = pickle.load(f)

In [2]:
import os 

# Get the list of all files and directories
path = "/net/atmos/data/cmip6-ng/tos/ann/g025"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

list_model = []
list_forcing = []

for idx, file in enumerate(dir_list):

    file_split = file.split("_")
    
    # extract model names
    model_name = file_split[2]
    forcing = file_split[3]
    run_name = file_split[4]
    
    list_model.append(model_name)
    list_forcing.append(forcing)
    
model_names = list(set(list_model))
forcing_names = list(set(list_forcing))

Files and directories in ' /net/atmos/data/cmip6-ng/tos/ann/g025 ' :


# Load the observations

In [3]:
import netCDF4 as netcdf

# define the file
file = '/net/h2o/climphys3/simondi/cope-analysis/data/erss/sst_annual_g050_mean_19812014_centered.nc'

# read the dataset
file2read = netcdf.Dataset(file,'r')

# load longitude, latitude and sst monthly means
lon = np.array(file2read.variables['lon'][:])
lat = np.array(file2read.variables['lat'][:])
sst = np.array(file2read.variables['sst'])

# define grid
lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')

# Preprocessing of the data: $(x_{i,t,m}^{p})_{i=1,\ldots,I, t=1,\ldots,T,m=1,\ldots,M, p=1,\ldots,d}$
## $i$: ensemble member (run) index
## $t$: time index
## $m$: model index
## $p$: grid cell index

#### Keep the model with at least 3 ensemble memebers and downscale the data from latitude 144 -> 36 with local averaging (to match with ensemble methods

In [4]:
import skimage

# first filter out the models that do not contain ensemble members 
dic_reduced_ssp585 = {}

for m in list(dic_ssp585.keys()):
    if len(dic_ssp585[m].keys()) > 2:
        dic_reduced_ssp585[m] = dic_ssp585[m].copy()
        for idx_i, i in enumerate(dic_ssp585[m].keys()):
            dic_reduced_ssp585[m][i] = skimage.transform.downscale_local_mean(dic_reduced_ssp585[m][i],(1,2,2))
            lat_size = dic_reduced_ssp585[m][i][0,:,:].shape[0]
            lon_size = dic_reduced_ssp585[m][i][0,:,:].shape[1]

######## Store Nan indices 

nan_idx = []
for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    for idx_i,i in enumerate(dic_reduced_ssp585[m].keys()):    

        nan_idx_tmp = list(np.where(np.isnan(dic_reduced_ssp585[m][i][0,:,:].ravel())==True)[0])        
        nan_idx = list(set(nan_idx) | set(nan_idx_tmp))

notnan_idx = list(set(list(range(lon_size*lat_size))) - set(nan_idx))

### 1) Compute anomalies: $\displaystyle \overline{x}_{i,t,m}^p = x_{i,t,m}^p - \frac{1}{t_{\mathrm{ref}}^f - t_{\mathrm{ref}}^s} \sum_{t= t_{\mathrm{ref}}^s}^{t_{\mathrm{ref}}^f} \sum_{i=1}^I x_{i,t,m}^p$

In [5]:
# second, for each model we compute the anomalies 
dic_processed_ssp585 = {}
time_period = 33

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_processed_ssp585[m] = dic_reduced_ssp585[m].copy()
    
    mean_ref_ensemble = 0
    y_tmp = np.zeros((len(dic_reduced_ssp585[m].keys()),time_period, lat_size*lon_size))
    
    for idx_i, i in enumerate(dic_reduced_ssp585[m].keys()):
        y_tmp[idx_i,:,:] = dic_reduced_ssp585[m][i][131:164,:,:].copy().reshape(time_period, lat_size*lon_size)
        y_tmp[idx_i,:,nan_idx] = float('nan')
           
        if idx_i == 0:
            mean_ref_ensemble = np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())
        else:
            mean_ref_ensemble += np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())

    for idx_i, i in enumerate(dic_processed_ssp585[m].keys()):
        dic_processed_ssp585[m][i] = y_tmp[idx_i,:,:] - mean_ref_ensemble

/tmp/ipykernel_46089/1144963586.py:16: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble = np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())
/tmp/ipykernel_46089/1144963586.py:18: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble += np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())


### 2) Compute the forced response: 
#### - Mean over space: $\displaystyle y_{i,t,m} = \frac{1}{P} \sum_{p=1}^P x_{i,t,m}^p$
#### - Mean over ensemble members: $\displaystyle \overline{y}_{t,m} = \frac{1}{I} \sum_{i=1}^I y_{i,t,m}$
#### - Set the mean to all the ensemble member forced responses: $y_{i,t,m} \colon= \overline{y}_{t,m}$
#### - Centering with respect to a given reference period: $\displaystyle y_{i,t,m} = y_{i,t,m} - \frac{1}{t_{\mathrm{ref}}^f - t_{\mathrm{ref}}^s} \sum_{t= t_{\mathrm{ref}}^s}^{t_{\mathrm{ref}}^f} \overline{y}_{t,m}$

In [6]:
# compute the forced response
dic_forced_response_ssp585 = dict({})

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_forced_response_ssp585[m] = dic_reduced_ssp585[m].copy()

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        
        y_tmp = dic_reduced_ssp585[m][i][131:164,:,:].copy().reshape(time_period, lat_size*lon_size)
        y_tmp[:,nan_idx] = float('nan')

        if idx_i == 0:
            mean_spatial_ensemble = np.nanmean(y_tmp,axis=1)/ len(dic_forced_response_ssp585[m].keys())
        else:
            mean_spatial_ensemble += np.nanmean(y_tmp,axis=1)/ len(dic_forced_response_ssp585[m].keys())

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):        
        dic_forced_response_ssp585[m][i] = mean_spatial_ensemble - np.nanmean(mean_spatial_ensemble)

In [7]:
y_forced_response = {}
x_predictor = {}

for idx_m,m in enumerate(dic_processed_ssp585.keys()):
    y_forced_response[m] = {}
    x_predictor[m] = {}

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):       
        y_forced_response[m][i] = dic_forced_response_ssp585[m][i]
        x_predictor[m][i] = dic_processed_ssp585[m][i]
        x_predictor[m][i][:,nan_idx] = float('nan')

## Now we can use the data to run some simple regression models

In [8]:
# compute the variance
variance_processed_ssp585 = {}
std_processed_ssp585 = {}
for idx_m,m in enumerate(x_predictor.keys()):
    variance_processed_ssp585[m] = {}
    arr_tmp = np.zeros((len(x_predictor[m].keys()),33))
    
    for idx_i, i in enumerate(list(x_predictor[m].keys())):
        arr_tmp[idx_i,:] = np.nanmean(x_predictor[m][i],axis=1)

    arr_tmp_values = np.zeros((len(x_predictor[m].keys()),33))
    for idx_i, i in enumerate(x_predictor[m].keys()):
        arr_tmp_values[idx_i,:] = (y_forced_response[m][i] - arr_tmp[idx_i,:])**2

    variance_processed_ssp585[m] = torch.nanmean(torch.from_numpy(arr_tmp_values),axis=0)

# Define training set

In [9]:
import torch 

# Data preprocessing
x_train = {}
y_train = {}

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    x_train[m] = {}
    y_train[m] = {}
    for idx_i, i in enumerate(dic_processed_ssp585[m].keys()):
        x_train[m][i] = torch.nan_to_num(torch.from_numpy(x_predictor[m][i])).to(torch.float64)
        y_train[m][i] = torch.from_numpy(y_forced_response[m][i]).to(torch.float64)


In [10]:
def train_ridge_regression(subset_runs,x,y,vars,lon_size,lat_size,lambda_,nbEpochs=200,verbose=True):
    """
    Given a model m, learn parameter β^m such that β^m = argmin_{β}(||y_m - X_m^T β||^2) ).

    Args:
        - x, y: training set and training target 
        - lon_size, lat_size: longitude and latitude grid size (Int)
        - lambda_: regularizer coefficient (float)
        - nbepochs: number of optimization steps (Int)
        - verbose: display logs (bool)
    """

    # define variable beta
    beta = torch.zeros(lon_size*lat_size).to(torch.float64)
    beta.requires_grad_(True)  
                          
    # define optimizer
    optimizer = torch.optim.Adam([beta],lr=1e-3)
            
    # --- optimization loop ---                
    for epoch in torch.arange(nbEpochs):      
                      
        optimizer.zero_grad()
        ############### Define loss function ##############
                    
        # first term: ||Y - β^T X||
        obj = 0.0
        for idx_r,r in enumerate(x.keys()):
            if r in subset_runs:
                obj += 0.5*torch.mean(((y[r] - torch.matmul(x[r],beta))**2/vars))
        obj += 0.5*lambda_*torch.norm(beta,p=2)**2
                    
        #define loss function
        loss = obj
                    
        # Use autograd to compute the backward pass. 
        loss.backward(retain_graph=True)               
        
        # take a step into optimal direction of parameters minimizing loss
        optimizer.step()       
        
        if(verbose==True):
            if(epoch % 10 == 0):
                print('Epoch ', epoch.detach().item(), 
                    ', loss=', loss.detach().item()
                    )
    return beta.detach().clone()

In [11]:
lambda_ = 1.0
model_name = 'CanESM5-1'
nbEpochs = 300
subset_runs = list(x_train[model_name].keys())


beta = train_ridge_regression(subset_runs, x_train[model_name],y_train[model_name],variance_processed_ssp585[model_name],\
                              lon_size,lat_size,\
                              lambda_,nbEpochs,True)

Epoch  0 , loss= 90.85145770790857
Epoch  10 , loss= 25.711608179570327
Epoch  20 , loss= 10.536570297433858
Epoch  30 , loss= 4.168216033974605
Epoch  40 , loss= 2.0931041105817325
Epoch  50 , loss= 2.13790295834562
Epoch  60 , loss= 1.7360182522964933
Epoch  70 , loss= 1.6208768729270715
Epoch  80 , loss= 1.4639389231629527
Epoch  90 , loss= 1.35913690544825
Epoch  100 , loss= 1.2748240459496512
Epoch  110 , loss= 1.1995785900265479
Epoch  120 , loss= 1.132509050071427
Epoch  130 , loss= 1.0720897321024132
Epoch  140 , loss= 1.0172998706228904
Epoch  150 , loss= 0.9672571053204059
Epoch  160 , loss= 0.9213660269746677
Epoch  170 , loss= 0.8791171197354066
Epoch  180 , loss= 0.8400651382923667
Epoch  190 , loss= 0.8038549778051282
Epoch  200 , loss= 0.7701821413114388
Epoch  210 , loss= 0.7387883130491862
Epoch  220 , loss= 0.7094506811140122
Epoch  230 , loss= 0.6819764396249004
Epoch  240 , loss= 0.6561974048897692
Epoch  250 , loss= 0.6319660445285885
Epoch  260 , loss= 0.609152222

In [12]:
def single_cross_validation(x,y,vars,lon_size,lat_size,lambda_,nbEpochs=200,verbose=True):
    """
    Cross validation procedure: LOO--> for a given model, train the ridge regression on all runs except one, and test on this one.

    Args:
        - x, y: training set and training target of a single model
        - lon_size, lat_size: longitude and latitude grid size (Int)
        - lambda_: regularizer coefficient (float)
        - nbepochs: number of optimization steps (Int)
        - verbose: display logs (bool)
    """
    # number of runs
    n_runs = len(x.keys())

    beta = {}
    y_pred = {}
    rmse = {}

    for idx_r, r in enumerate(x.keys()):
        print("Remove another run ", r)
        
        # get list of run names 
        list_runs = list(x.keys())
        
        # remove run r
        list_runs.remove(r)

        # train model 
        beta[r] = train_ridge_regression(list_runs,x,y,vars,lon_size,lat_size,lambda_,nbEpochs,verbose)

        # prediction
        y_pred[r] = torch.matmul(x[r],beta[r]) 

        # rmse 
        rmse[r] = torch.sqrt(torch.mean((y[r] - y_pred[r])**2/vars))


    return beta, y_pred, rmse

In [14]:
def all_models_cross_validation(x,y,vars,lon_size,lat_size,lambda_range,nbEpochs=200,verbose=True):
    """
    Cross validation procedure: LOO--> for each model, train the ridge regression on all runs except one, and test on this one.

    Args:
        - x, y: training set and training target of a single model
        - lon_size, lat_size: longitude and latitude grid size (Int)
        - lambda_range: set of regularizer coefficients to test (float)
        - nbepochs: number of optimization steps (Int)
        - verbose: display logs (bool)
    """
    beta = {}
    rmse = {}

    for idx_m, m in enumerate(x.keys()):
    
        beta[m] = {}
        rmse[m] = {}
        for idx_lambda, lambda_ in enumerate(lambda_range):
            
            print("Start new model: ", m)
            beta[m][lambda_],y_pred, rmse[m][lambda_] = single_cross_validation(x[m],y[m],vars[m],\
                                                                         lon_size,lat_size,
                                                                         lambda_,nbEpochs,verbose)

    return beta, rmse

In [15]:
lambda_range = torch.tensor([10.0])

beta, rmse = all_models_cross_validation(x_train,y_train,variance_processed_ssp585,\
                                       lon_size,lat_size,\
                                       lambda_range,nbEpochs=200,verbose=False)


with open('results/beta_individual.npy', 'wb') as f:
    np.save(f, beta)

with open('results/rmse_individual.npy', 'wb') as f:
    np.save(f, rmse)

Start new model:  CanESM5-1
Remove another run  r10i1p1f1
Remove another run  r10i1p2f1
Remove another run  r1i1p1f1
Remove another run  r1i1p2f1
Remove another run  r2i1p1f1
Remove another run  r3i1p1f1
Remove another run  r3i1p2f1
Remove another run  r4i1p1f1
Remove another run  r4i1p2f1
Remove another run  r5i1p1f1
Remove another run  r6i1p1f1
Remove another run  r6i1p2f1
Remove another run  r7i1p1f1
Remove another run  r7i1p2f1
Remove another run  r8i1p1f1
Remove another run  r8i1p2f1
Remove another run  r9i1p1f1
Remove another run  r9i1p2f1
Start new model:  CNRM-ESM2-1
Remove another run  r1i1p1f2
Remove another run  r2i1p1f2
Remove another run  r3i1p1f2
Remove another run  r4i1p1f2
Remove another run  r5i1p1f2
Start new model:  FIO-ESM-2-0
Remove another run  r1i1p1f1
Remove another run  r2i1p1f1
Remove another run  r3i1p1f1
Start new model:  GISS-E2-2-G
Remove another run  r1i1p3f1
Remove another run  r2i1p3f1
Remove another run  r3i1p3f1
Remove another run  r4i1p3f1
Remove ano

In [ ]:
# now we do the plotting

In [ ]:
best_lambda = {}
best_rmse = {}

for idx_m, m in enumerate(x_train.keys()):
    test_rmse = np.zeros(lambda_range.shape[0])
    for idx_lambda, lambda_ in enumerate(lambda_range):
        test_rmse[idx_lambda] = np.array(list(rmse[m][idx_lambda].values())).mean()

    # find mininum
    best_rmse[m] = np.min(test_rmse)
    best_lambda[m] = lambda_range[np.argmin(test_rmse)]

In [ ]:
def plot_betas(x,y,vars,lon_size,lat_size,lambda_):

    beta = torch.zeros(len(x.keys()),lon_size*lat_size)

    for idx_m,m in enumerate(x.keys()):
        # beta[idx_m,:] =  ridge_estimator(x[m],y[m],vars[m],lambda_)
        beta[idx_m,:] =  train_single_ridge_regression(x[m],y[m],vars[m],lon_size,lat_size,lambda_,nbEpochs=100,verbose=True)


    # plot the beta map of each mode
    fig, axs = plt.subplots(6,5, figsize=(15,10), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = 2.0, wspace=1.0)

    axs = axs.ravel()
    
    for idx_m, m in enumerate(x.keys()):
        
        beta_tmp = beta[idx_m,:].detach().clone()
        beta_tmp[nans_idx] = float('nan')
        beta_tmp = beta_tmp.detach().numpy().reshape(lat_size,lon_size)

        axs[idx_m].set_title(m+ ' ('+ str(len(dic_processed_ssp585[m].keys())) +') ')
        im0 = axs[idx_m].pcolormesh(lon_grid,lat_grid,beta_tmp,vmin=-0.00,vmax = 0.01)

    plt.colorbar(im0, ax=axs[idx_m], shrink=0.5)

    for i in range(len(x.keys()),30):
        fig.delaxes(axs[i])

    fig.tight_layout()
    plt.show()

    return beta

In [ ]:
# plot_betas_descent(x_train,y_train,vars_ssp585,grid_lon_size,grid_lat_size,1.0,nbEpochs=5000,verbose=False)

# Analysis of the betas

### 1) PCA on the betas

In [ ]:
# run with PCA with pytorch
U,S,V = torch.pca_lowrank(beta, q=6, center=False, niter=10)
proj_first_comp = torch.matmul(beta, V[:, :6])

In [ ]:
fig = plt.figure()
plt.plot(S**2/25)
plt.ylim((0.0,0.002))
plt.xlim((0.0,6))
plt.title('Eigenvalues')
plt.show()

In [ ]:
comp_x = 0
comp_y = 1

fig, ax = plt.subplots()
ax.scatter(proj_first_comp[:,comp_x],proj_first_comp[:,comp_y])

for idx_m, m in enumerate(dic_reduced_ssp585.keys()):
    ax.annotate(m, (proj_first_comp[idx_m,comp_x]+.003, proj_first_comp[idx_m,comp_y]+.003))

plt.show()

In [ ]:
# plot the beta map of each mode
fig, axs = plt.subplots(2,3, figsize=(15,10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 2.0, wspace=1.0)

axs = axs.ravel()
    
for k in range(6):
        
    beta_tmp = V[:,k].detach().clone()
    beta_tmp[nans_idx] = float('nan')
    beta_tmp = beta_tmp.detach().numpy().reshape(grid_lat_size,grid_lon_size)

    axs[k].set_title('Component '+ str(k))
    im0 = axs[k].pcolormesh(lon_grid,lat_grid,beta_tmp,vmin=-0.00,vmax = 0.1)

    plt.colorbar(im0, ax=axs[k], shrink=0.5)

fig.tight_layout()
plt.show()

### 2) Hierarchical clustering (within cluster variance based)

#### Ward based hierarchical clustering: minimize the total within cluster variance 

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list

models = list(dic_reduced_ssp585.keys())
Z1 = linkage(beta.detach().numpy(), 'ward')
leaves_tmp = leaves_list(Z1)

labels_tmp =  [models[int(i)] for i in leaves_tmp] 


fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z1,labels = models)

#### plot the model variance to check how it impacts the clustering

In [ ]:
# compute the coefficient using soft max
M = len(list(dic_reduced_ssp585.keys()))
gamma = torch.zeros(M)
ordered_betas = [models[int(i)] for i in dn['leaves']]

for idx,i in enumerate(dn['leaves']):
    m = models[int(i)] 
    gamma[idx] = vars_ssp585[m]

# plot the model contributions
fig, ax = plt.subplots()
models = list(dic_reduced_ssp585.keys())
weights = list(gamma.detach().numpy())

ax.bar(models, weights,label='Model variance')
ax.set_ylabel(r'Internal variances')
ax.set_title('cmip6 models')
ax.legend()
ax.set_xticklabels(ordered_betas, rotation=-90)
plt.tight_layout()
plt.show()

### Plot the betas with respect to a given ordering

In [ ]:
# plot the beta maps in the leaf-based ordering
fig, axs = plt.subplots(6,5, figsize=(15,10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 2.0, wspace=1.0)

axs = axs.ravel()
    
for idx,i in enumerate(dn['leaves']):

    m = models[int(i)] 
    beta_tmp = beta[int(i),:].detach().clone()
    beta_tmp[nans_idx] = float('nan')
    beta_tmp = beta_tmp.detach().numpy().reshape(grid_lat_size,grid_lon_size)

    axs[idx].set_title(m)
    im0 = axs[idx].pcolormesh(lon_grid,lat_grid,beta_tmp,vmin=-0.00,vmax = 0.01)

plt.colorbar(im0, ax=axs[idx], shrink=0.5)

for i in range(len(dic_reduced_ssp585.keys()),30):
    fig.delaxes(axs[i])

fig.tight_layout()
plt.show()